In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
import requests
import time
import os

In [13]:
download_path="../dataset/onepiece/"

In [15]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(options=options, executable_path="../.bin/chromedriver")
driver.implicitly_wait(0.5)
driver.get("https://mangasee123.com/manga/One-Piece-Digital-Colored-Comics")

In [17]:
driver.find_element_by_class_name('ShowAllChapters').click()
chapterLinks = driver.find_elements_by_css_selector('a.ChapterLink')

In [18]:
links = list([{'name':elem.text.split("\n")[0],'link':elem.get_attribute('href')} for elem in chapterLinks])
print(f"Found {len(links)} available chapters")

Found 942 available chapters


In [19]:
def find_pages():
    btn_modal = driver.find_element_by_css_selector("body > div.MainContainer > div.container.DesktopNav.top-10 > div > div.Column.col-lg-2.col-6.ng-scope > button")
    btn_modal.click()
    time.sleep(0.5)
    row = driver.find_element_by_css_selector("#PageModal > div > div > div.modal-body > div")
    buttons = row.find_elements_by_tag_name("button")
    numbers = [int(button.text) for button in buttons]
    driver.find_element_by_css_selector("#PageModal > div > div > div.modal-footer > button").click()
    return max(numbers)

In [21]:
for chapter in tqdm(links):    
    if not os.path.exists(download_path+chapter.get("name")):
        os.mkdir(download_path+chapter.get("name"))
        # go to chapter
        driver.get(chapter.get("link"))    
        # extract the number of pages
        try:
            pages = find_pages()
        except:
            pages = 15
        # extract max
        pages = pages if pages<15 else 15
        # download the pages
        for page in range(pages):
            time.sleep(1)
            img = driver.find_element_by_css_selector("img.img-fluid")
            src = img.get_attribute("src")   
            r = requests.get(src)
            with open(download_path+chapter.get("name")+"/"+str(page)+'.png', 'wb') as outfile:
                outfile.write(r.content)
            img.click()

100%|██████████| 942/942 [00:25<00:00, 36.79it/s]  


In [22]:
driver.quit()